In [1]:
import time
import sys
# from graph import graph # my own code for graph object
import heapq
from collections import deque,defaultdict
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable # for linear programming

In [2]:
from collections import deque,defaultdict
class vertex:
    def __init__(self,node):
        self.id = node
        self.adjacent = set()

    def __str__(self):
        # for print out result
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor):
    
        self.adjacent.add(neighbor)

    def remove_neighbor(self, neighbor):
        if neighbor in self.adjacent:
            self.adjacent.remove(neighbor)

    def is_connected(self,neighbor):
        return neighbor in self.adjacent

    def get_connections(self):
        return self.adjacent



class graph:
    # unweighted undirected graph
    # can be connected or not
    def __init__(self):
        self.vert_dict = {} # vertex_id (int) : vertex
        self.num_vertices = 0
        self.num_edges = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self,node):
        self.num_vertices += 1
        new_vertex = vertex(node)
        self.vert_dict[node] = new_vertex

    def get_vertex(self,node):
        if node in self.vert_dict:
            return self.vert_dict[node]
        else:
            return None

    def add_edge(self, frm, to):
        # for new vertices
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        if not self.vert_dict[frm].is_connected(self.vert_dict[to]):
            self.num_edges += 1

        self.vert_dict[frm].add_neighbor(self.vert_dict[to])
        self.vert_dict[to].add_neighbor(self.vert_dict[frm])


    def remove_edge(self, frm, to):
        self.vert_dict[frm].remove_neighbor(self.vert_dict[to])
        self.vert_dict[to].remove_neighbor(self.vert_dict[frm])
        self.num_edges -= 1
    
    def remove_vertex(self,node,inplace = False):
        
        G = self.copy() if not inplace else self
        
        for neighbor in list(G.vert_dict[node].get_connections()):
            G.remove_edge(node, neighbor.id)
            if not neighbor.get_connections():
                del G.vert_dict[neighbor.id]
        del G.vert_dict[node]
        
        return G
            

    def get_vertices(self):
        # return a list of ints, the id of vertices
        return list(self.vert_dict.keys())
    
    def get_vertex_degree(self,node):
        return len(self.get_vertex(node).adjacent)
    
    def get_max_degree_vertex(self):
        return max(self.vert_dict, key = lambda x: len(self.get_vertex(x).get_connections()))
    
    def copy(self):
        copy = graph()
        for v in self.vert_dict:
            for u in self.vert_dict[v].get_connections():
                copy.add_edge(v,u.id)
        return copy


In [671]:
G.get_max_degree_vertex()

34

In [3]:
# read data and construct a graph object
def parse_edges(filename):
    # parse edges from graph file to create your graph object
    # filename: string of the filename
    f = open(filename, "r")
    n_vertices, n_edges, _ = f.readline().split(' ')
    n_vertices, n_edges = int(n_vertices), int(n_edges)

    G = graph() # create a graph

    # add edges to the graph
    for i in range(1,n_vertices+1):
        newline = f.readline()
        newline = newline.rstrip('\n')
        newline = newline.strip()
        
        if not newline:
            G.add_vertex(i)
        else:
            neighbors = list(map(int, newline.split(' ')))

            for neighbor in neighbors:
                if neighbor > i:
                    G.add_edge(i, neighbor)
                
    return G  

In [4]:
# G = parse_edges('../DATA/karate.graph')
# G = parse_edges('../DATA/netscience.graph')
G = parse_edges('../DATA/dummy2.graph')

In [626]:
for v in G:
    print(v)

1 adjacent: [2, 3]
2 adjacent: [4, 1, 3]
3 adjacent: [2, 1, 5]
4 adjacent: [2, 6, 5]
5 adjacent: [4, 3]
6 adjacent: [4]


In [619]:
G1 = G.remove_vertex(4)

## 2-approximation Algorithm to find a lower bound

In [4]:
def approx2(G):
    # G: a graph object
    # return: the number of vertices of a 2-approximation solution to the min vertex cover problem of G
    S = set()
    for v in G:
        if v in S:
            continue
        for neighbor in v.get_connections():
            
            if neighbor.id < v.id:
                continue
            if v.id not in S and neighbor.id not in S:
                S.add(v.id)
                S.add(neighbor.id)
    return len(S)

Since $|S| \leq 2 OPT(G)$, we know $|S|/2$ is a lower bound of $OPT(G)$.

In [6]:
LB = approx2(G) // 2
n = G.num_vertices

In [7]:
LB

3

## Branch-and-Bound

$C'$: partial vertex cover of $G = (V, E)$

$G' = (V', E')$: subgraph not covered by $C'$

$V' = V - C' - \{u | \forall (u,v) \in E,  v \in C'\}$

$E' = \{(u,v) \in E | u, v \in V' \}$

LB = $|C'|$ + LB of the VC for $G'$

* If we have $V'$ empty, we found a vertex cover
* LB of the VC for $G'$ can be found using approx2($G'$)
* The problem is how to do recursion and how to find and update $G'$

In [5]:
def Find_Vprime(G, Cprime):
    'Cprime: a list of numbers in {1, 2, ..., n}'
    'Return: a set of vertices Vprime of the graph Gprime'
    Vprime = set()
#     Cprime = set(Cprime)
    n = G.num_vertices
    for node in range(1, n+1):
        if node in Cprime:
            continue
            
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id not in Cprime:
                Vprime.add(node)
                break
    return Vprime

In [6]:
def Find_LowerBound(V_prime):
    'Find lower bound by linear programming, slow'
    'Cprime: a list of vertices of a vertex cover'
   
    V_prime = {i:x for i,x in zip(V_prime, range(len(V_prime)))} # nodes: variable index
    
    # Prepare for Linear Regression
    obj = [1] * len(V_prime) # coeff of objective function
    bnd = [(0,1)] * len(V_prime) # bounds for each variable

    # Linear constraints, has to be <= 
    lhs_ineq = []
    rhs_ineq = []

    for node in V_prime:
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id > node and neighbor.id in V_prime:
                new_constraint = [0]*len(V_prime)
                new_constraint[V_prime[node]] = -1
                new_constraint[V_prime[neighbor.id]] = -1
                lhs_ineq.append(new_constraint)
                rhs_ineq.append(-1)

    # solve the linear programming problem
    print('Number of constraints:', len(lhs_ineq))
    opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd,
                  method="revised simplex")
    
    # lowerbound is half of the optimal of the linear optimization problem
    lowerbound = opt.fun // 2
    
    return lowerbound

In [7]:
def Gprime(G, V_prime):
    Gprime = graph()
    for node in V_prime:
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id > node and neighbor.id in V_prime:
                Gprime.add_edge(node,neighbor.id)
    return Gprime

In [8]:
G = parse_edges('../DATA/karate.graph')

In [679]:
## This code is working, but not very efficient, trying to improve the speed using the following cell
class node:
    def __init__(self,id, parent,lb,state):
        self.id = id
        self.parent = parent
        self.lb = lb
        self.state = state
#         self.degree = degree
    
    def __lt__(self, other):
        return self.lb > other.lb
        
# dummy = node(0,None,LB)       

n = G.num_vertices
opt_num = n
vertices = G.get_vertices()
vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent))
opt_cover = vertices
LB = approx2(G) / 2
vertices_order = {vertex: i for i, vertex in enumerate(vertices)}

num_uncov_edges0 = G.num_edges
num_uncov_edges1 = G.num_edges - G.get_vertex_degree(vertices[0])
pqueue = [(num_uncov_edges0, node(vertices[0],None,LB,0)),(num_uncov_edges1,node(vertices[0],None,LB,1))]
heapq.heapify(pqueue)

while pqueue:

    num_uncov_edges,Dnode = heapq.heappop(pqueue)
    parent = Dnode.parent
    node_id = Dnode.id   
        
        
    Cprime = set([Dnode.id]) if Dnode.state else set()
    explored = set() if Dnode.state else set([Dnode.id])
    while parent:
#         print([parent.id, parent.state],end='')
        if parent.state:
            Cprime.add(parent.id)
        else:
            explored.add(parent.id)
        parent = parent.parent
#     print('')
#     print('Cprime',Cprime) 
    V_prime = Find_Vprime(G,Cprime)
    G_prime = Gprime(G, V_prime)
    
    new_node_id = find_next(V_prime, G_prime, explored)
    
    if not new_node_id:
        continue

#     new_node_id = vertices[vertices_order[node_id]+1]
#         print(new_node_id)
    cover_size = len(Cprime) + 1  
    new_cover = Cprime.union(set([new_node_id]))
#         print(new_cover)
    # get Vprime
#         print('new cover',new_cover)
    G_prime = G_prime.remove_vertex(new_node_id,inplace = True)
    V_prime = G_prime.get_vertices()
#         print('V prime',V_prime)
    if not V_prime:
        # solution found
        if cover_size < opt_num:
            opt_num = cover_size
            opt_cover = new_cover 
            print('Optimal:', opt_num)
            continue

    
    LowerBound = cover_size + approx2(G_prime)//2

    if LowerBound < opt_num:
        new_node1 = node(new_node_id, Dnode, LowerBound,1)
        heapq.heappush(pqueue, (G_prime.num_edges, new_node1))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')


    if Dnode.lb < opt_num:
        new_node0 = node(new_node_id, Dnode, Dnode.lb,0)
        heapq.heappush(pqueue, (num_uncov_edges, new_node0))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')

        

Optimal: 7374


KeyboardInterrupt: 

In [666]:
G = parse_edges('../DATA/karate.graph')

In [667]:
approx2(G)

22

In [13]:
## This code is working, but not very efficient, trying to improve the speed using the following cell
class node:
    def __init__(self,id, parent,lb,state):
        self.id = id
        self.parent = parent
        self.lb = lb
        self.state = state
#         self.degree = degree
    
    def __lt__(self, other):
        return self.lb > other.lb
        
# dummy = node(0,None,LB)       

n = G.num_vertices
opt_num = n
vertices = G.get_vertices()
vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent))
opt_cover = vertices
LB = approx2(G) / 2
vertices_order = {vertex: i for i, vertex in enumerate(vertices)}

num_uncov_edges0 = G.num_edges
num_uncov_edges1 = G.num_edges - G.get_vertex_degree(vertices[0])
pqueue = [(num_uncov_edges0, node(vertices[0],None,LB,0)),(num_uncov_edges1,node(vertices[0],None,LB,1))]
heapq.heapify(pqueue)
new_node0, new_node1 = None, None
i, j = 1,1
while pqueue:

    num_uncov_edges,Dnode = heapq.heappop(pqueue)
    parent = Dnode.parent
    node_id = Dnode.id   
    
#     print(Dnode, new_node1, new_node0)
    if Dnode == new_node1:
#         print('a1')
        Cprime.add(new_node_id)
    elif Dnode == new_node0:
#         print('a2')
        explored.add(new_node_id)
    else:
        j += 1
        print(j/i)
        Cprime = set([Dnode.id]) if Dnode.state else set()
        explored = set() if Dnode.state else set([Dnode.id])
        while parent:
#             print([parent.id, parent.state],end='')
            if parent.state:
                Cprime.add(parent.id)
            else:
                explored.add(parent.id)
            parent = parent.parent
    i += 1
#     print('')
#     print('Cprime',Cprime) 
    V_prime = Find_Vprime(G,Cprime)
    G_prime = Gprime(G, V_prime)
    
    new_node_id = find_next(V_prime, G_prime, explored)
    
    if not new_node_id:
        continue

#     new_node_id = vertices[vertices_order[node_id]+1]
#         print(new_node_id)
    cover_size = len(Cprime) + 1  
    new_cover = Cprime.union(set([new_node_id]))
#         print(new_cover)
    # get Vprime
#         print('new cover',new_cover)
    G_prime = G_prime.remove_vertex(new_node_id,inplace = True)
    V_prime = G_prime.get_vertices()
#         print('V prime',V_prime)
    if not V_prime:
        # solution found
        if cover_size < opt_num:
            opt_num = cover_size
            opt_cover = new_cover 
            print('Optimal:', opt_num)
            continue

    
    LowerBound = cover_size + approx2(G_prime)//2

    if LowerBound < opt_num:
        new_node1 = node(new_node_id, Dnode, LowerBound,1)
        heapq.heappush(pqueue, (G_prime.num_edges, new_node1))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')


    if Dnode.lb < opt_num:
        new_node0 = node(new_node_id, Dnode, Dnode.lb,0)
        heapq.heappush(pqueue, (num_uncov_edges, new_node0))
#         print('new decision node added')
#     else:
#         print('new decision node pruned')


        

2.0
Optimal: 14
0.2
0.25
0.29411764705882354
0.2222222222222222
0.22580645161290322
0.19047619047619047
0.20454545454545456
0.19607843137254902
0.2
0.2033898305084746
0.19117647058823528
0.1917808219178082
0.19230769230769232
0.1927710843373494
0.19318181818181818
0.1875
0.18446601941747573
0.17699115044247787
0.1794871794871795
0.18181818181818182
0.18548387096774194
0.18604651162790697
0.18796992481203006
0.18705035971223022
0.1875
0.19047619047619047
0.18354430379746836
0.18072289156626506
0.1791907514450867
0.1797752808988764
0.18032786885245902
0.1827956989247312
0.18617021276595744
0.18652849740932642
0.18686868686868688
0.1890547263681592
0.19117647058823528
0.19047619047619047
0.19248826291079812
0.1935483870967742
0.19457013574660634
0.19555555555555557
0.1965065502183406
0.19658119658119658
0.19747899159663865
0.19834710743801653
0.2
0.20080321285140562
0.2015810276679842
0.20233463035019456
0.20384615384615384
0.1978021978021978
0.19855595667870035
0.199288256227758
0.199300

0.17280622941000298
0.17274357441721458
0.1726811810319117
0.17277330950253203
0.17291666666666666
0.17275155832591274
0.17263843648208468
0.17272996154983733
0.17261729123635292
0.17270851753610375
0.17285041224970554
0.172737955346651
0.17267663441805922
0.17261556465769456
0.1726051401869159
0.1727962638645651
0.1722335172814406
0.1719247467438495
0.17176674364896075
0.17190654744736084
0.17194700460829493
0.1714941210209349
0.17163323782234957
0.17167381974248927
0.17122507122507122
0.17121729237770195
0.17106382978723406
0.1711533012184755
0.17071106094808128
0.17065615319628274
0.17036205444849845
0.17049915872125632
0.17030201342281878
0.17048630519843488
0.1704323570432357
0.1702364394993046
0.17041979427300527
0.17027176927343315
0.17026578073089702
0.17044825677919204
0.17020689655172414
0.17010734929810073
0.1701952158372285
0.16967706622879036
0.1694406548431105
0.1694361209479706
0.16920152091254753
0.16919739696312364
0.16933080465998374
0.1690977849810913
0.1691394658753

0.162144597887896
0.16217532467532467
0.16202203499675957
0.1621315192743764
0.16197866149369544
0.16200936843805525
0.1620923474329997
0.16207063376874697
0.16207507652650233
0.16205342774380432
0.1621360784944507
0.16211439588688947
0.162144806550008
0.16222721437740692
0.16225749559082892
0.16231373177375422
0.1623959000640615
0.1621405750798722
0.16219667943805874
0.16227860220201054
0.16217938505655569
0.1621836702212319
0.162291169451074
0.16229534255285327
0.16229950770208035
0.16238095238095238
0.16217928413050364
0.16218354430379747
0.16221343873517788
0.1623202149517939
0.16209397666351308
0.16197072249331024
0.16207710464201416
0.1621069182389937
0.1620094191522763
0.16206463759021023
0.16189134178800688
0.16194648724769206
0.161875
0.16193004372267333
0.16198501872659177
0.16206520043674932
0.16204425054534122
0.1620990345686702
0.16210329807093965
0.16215796019900497
0.16218735435762002
0.16224188790560473
0.162346732888406
0.16240111679851094
0.16248062015503875
0.1622583

0.15972548650935414
0.15968438850272404
0.15965834428383704
0.15966228893058163
0.15969617404351089
0.15962529274004683
0.1596143044373713
0.15964813775032752
0.15959237097980553
0.1596560104692466
0.1596002988419873
0.15966386554621848
0.15963799216271693
0.15962703962703964
0.15955663189269748
0.1595160539785947
0.15952004464700958
0.1594647337608029
0.15945393759286774
0.15947275596398403
0.15953596287703017
0.15948076031525266
0.15945520244602984
0.15941492316237735
0.15943370037938373
0.15948196114708604
0.1595449500554939
0.15951940850277263
0.15950863581786276
0.15952732644017725
0.15956072351421188
0.15960882000184518
0.15955387593326573
0.15955780746199907
0.15957642725598525
0.15960972017673047
0.1596576792122941
0.15970561177552897
0.15966541042375218
0.15965460224141098
0.15964380795005967
0.15961838363452893
0.15962226093334556
0.1596407624633431
0.15967387321363136
0.1597069597069597
0.15969616546170037
0.1596999908533797
0.1597184128725544
0.15975141656004388
0.159696859

0.15965328196769893
0.1597037037037037
0.1596949729769749
0.15970988750740084
0.159712975292203
0.15971606033717836
0.15967193734298804
0.1597222222222222
0.15973709474927997
0.15971658424976012
0.15973144459200236
0.1597698605886258
0.159820045726086
0.15977595990861523
0.15981432360742706
0.15986444673640784
0.15984391105875423
0.15985868845219695
0.15988521815907586
0.15993526079599793
0.15990295544772828
0.1598941876699243
0.15989717223650385
0.1599471249173827
0.15992660550458715
0.15994130594277328
0.1599442651804048
0.15995894729125432
0.15998534261634298
0.16003517256539898
0.15992095440240064
0.1598888238736103
0.15991517987715706
0.15987140143212042
0.15987437919953257
0.15985401459854015
0.15978704784130687
0.1597551198892209
0.1597116014856893
0.15970301353908867
0.1596944343397599
0.15972070696050622
0.1597585805700989
0.1597964376590331
0.159706587261239
0.15967484395412979
0.15970105935277898
0.15963462374945628
0.15960298485836413
0.15960605402273878
0.15963223050749295

...

0.1365439717875248
0.1365441590973632
0.13654145280851732
0.13653932546174255
0.13653835558964753
0.136537964410881
0.1365387304694949
0.1365406536966959
0.13654315546816734
0.1365392927058704
0.13654005864307384
0.13654198162774142
0.13654506164993008
0.13654235622103025
0.1365431220024743
0.1365450447396963
0.1365429180276475
0.1365425268268298
0.13654329250985991
0.13654463657852597
0.13654771587366138
0.13654616772706862
0.13654577646749858
0.13654654196740199
0.13654962090728112
0.1365486512014028
0.1365505732667522
0.1365524952669764
0.13655557390866083
0.1365551823883922
0.13655710419657718
0.13655960427571953
0.1365597909634906
0.13656055594845365
0.1365630558143474
0.13656439899214465
0.1365663204163473
0.1365688200819811
0.13657131968410577
0.13656803631313302
0.13656648798787355
0.13656551800056735
0.13656512627278308
0.1365670472779309
0.13656781185834338
0.13656973273668657
0.1365722317240445
0.13657010536653924
0.1365691354261912
0.13656874365052488
0.13656950805516282
0.

0.13683562641834915
0.13683866753755716
0.1368388439267636
0.13684131197508018
0.1368443528938907
0.13684510202885444
0.13684756988491023
0.13684946475146634
0.13685193248040725
0.13685440014736233
0.13685056576286939
0.13684845034576293
0.13685149061927046
0.1368510937539242
0.1368518425502704
0.13685373694079828
0.13685563126789332
0.1368580983513241
0.13685483735958348
0.136855585967908
0.1368569072958823
0.13685421941369713
0.1368566861122038
0.13685457109499558
0.13685761033765603
0.13685606805561948
0.136856243828142
0.13685584695978176
0.13685831321469966
0.1368590615259095
0.13686095502679815
0.1368639937074026
0.13686588707821684
0.1368683530110703
0.136863375084716
0.1368612604321362
0.13686372611358388
0.13686676427525624
0.13686350459920607
0.13686310761238293
0.13686328311262433
0.13686460350589544
0.13686706875350294
0.13687010640433647
0.13686741948975323
0.13687045695979122
0.1368683425827956
0.13686794551712314
0.1368709827160081
0.136869440855089
0.13687076081593055
0

0.13681362196409716
0.13680984631629312
0.13680832810901772
0.13680286052467697
0.13680303530188057
0.13679869688034804
0.13679887176188238
0.1368001748403139
0.13679583707869802
0.13679431992479343
0.13679111092786905
0.13678959411267547
0.13678525786370946
0.13678317757779876
0.13678166131472314
0.1367818366186036
0.13678313953248722
0.13678557001887834
0.13678800044516073
0.13678310149407522
0.13678327673184082
0.1367800696490758
0.1367768629099391
0.13677591102210024
0.13677721368761125
0.13677400745822774
0.1367719286679632
0.13677210417670982
0.1367728431990573
0.13677245516383277
0.1367726306422109
0.13677449655581556
0.13676847404282666
0.136765832639913
0.1367631914981258
0.13676224055622604
0.13676241628429975
0.13676315529341987
0.1367588249103197
0.1367567478523009
0.13675410781204958
0.13675259430077416
0.13675220714191594
0.13675238310926272
0.1367497436868739
0.1367516088787875
0.13675065876152834
0.13675027173018017
0.1367532627938573
0.13675287569267758
0.1367541775935

0.13648850742872004
0.13648813394018205
0.13648609660653985
0.1364857232252061
0.13648424072260665
0.13648386742890575
0.13648404869090378
0.13648312091430242
0.13648330218574795
0.13648403792870897
0.1364853281116063
0.13648828167846827
0.13648790832016444
0.13649086174607883
0.13649104277531787
0.13649233269015945
0.1364930681084243
0.1364949123626541
0.13649564768817707
0.1364974918255854
0.13649989033217166
0.13650228878030551
0.1364980342463528
0.13649655194821017
0.13649784145781807
0.13649525062641374
0.13649709438084526
0.1364994923857868
0.13649911881035337
0.13650040810028302
0.1365000345148231
0.13649688976953436
0.13649817896861005
0.13650002233112593
0.13650241985253517
0.13650038366753686
0.1365016726753061
0.13650406999167733
0.1365031422029522
0.13650387691308408
0.13650571987626556
0.13650811692924733
0.13650552671210356
0.13650515309083971
0.13650367135208855
0.1365027437737442
0.13650514045434067
0.13650476689111032
0.13650771747115334
0.13650845187605268
0.136510848

KeyboardInterrupt: 

In [14]:
i

248781

In [15]:
j

33963

In [10]:
def find_next(V_prime, G_prime, Explored):
    for v in sorted(V_prime, key = lambda x: - G_prime.get_vertex_degree(x)):
        if v not in Explored:
            return v
        

In [559]:
class node:
    def __init__(self,id, parent,lb):
        self.id = id
        self.parent = parent
        self.lb = lb
#         self.state = state
    
    def __lt__(self, other):
        return self.lb < other.lb
        
# dummy = node(0,None,LB)       

n = G.num_vertices
opt_num = n
vertices = G.get_vertices()
vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent))
opt_cover = vertices
# n = 15
# vertices = [34,1,33,2,3,4,6,32,24,5,25,7,9,30,8]
vertices_order = {vertex: i for vertex, i in zip(vertices,range(n))}
pqueue = [(n-1, vertices_order[vertex_id], node(vertex_id,None,LB)) for vertex_id in vertices]
heapq.heapify(pqueue)
while pqueue:
    _, _, Dnode = heapq.heappop(pqueue)
#     print('vertex_id:', Dnode.id)
    parent = Dnode.parent
    node_id = Dnode.id   
    # get Cprime
    Cprime = set([Dnode.id]) #if parent else []
    while parent:
        Cprime.add(parent.id)
        parent = parent.parent
#     print('Cprime',Cprime) 
#     print('pqueue', [item[2].id for item in pqueue])
    for i in range(vertices_order[node_id]+1, n): 
#         print(i)
        new_node_id = vertices[i]
#         print(new_node_id)
        cover_size = len(Cprime) + 1  
        new_cover = Cprime.union(set([new_node_id]))
#         print(new_cover)
        # get Vprime
#         print('new cover',new_cover)
        V_prime = Find_Vprime(G,new_cover)
#         print('V prime',V_prime)
        if not V_prime:
            # solution found
            if cover_size < opt_num:
                opt_num = cover_size
                opt_cover = new_cover 
                print('Optimal:', opt_num, opt_cover)
                continue
        else:
            
            G_prime = Gprime(V_prime)
            LowerBound = cover_size + approx2(G_prime)//2
            
            if LowerBound < opt_num:
                new_node = node(new_node_id, Dnode, LowerBound)
                heapq.heappush(pqueue, (n-cover_size, i, new_node))
#                 print('new covered added:', new_cover)
#                 print(new_node.id, 'added as a child of ', node_id)
#             else:
#                 print('new cover pruned:', new_cover)

#                 print('prune new cover with cover size =', cover_size, 'lower bound = ', LowerBound,'optimal conver cover size:', opt_num)
                        

Optimal: 18 {1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 24, 26, 28, 30, 31, 32, 33, 34}
Optimal: 17 {1, 2, 3, 4, 5, 6, 7, 8, 9, 14, 24, 25, 30, 31, 32, 33, 34}
Optimal: 16 {32, 33, 2, 3, 4, 5, 6, 7, 8, 9, 1, 34, 14, 24, 25, 30}
Optimal: 15 {32, 33, 2, 3, 4, 5, 6, 7, 1, 9, 34, 14, 24, 25, 30}
Optimal: 14 {32, 33, 2, 3, 4, 5, 6, 7, 1, 9, 34, 24, 25, 30}


KeyboardInterrupt: 

In [517]:
Cprime = set([34,1,33,2,3,4,6,32,24,5,25,7,9,30])
sorted(Cprime)

[1, 2, 3, 4, 5, 6, 7, 9, 24, 25, 30, 32, 33, 34]

In [507]:
V_prime = Find_Vprime(G,Cprime)
G_prime = Gprime(V_prime)

In [510]:
LowerBound = len(Cprime) + approx2(G_prime)
LowerBound

14

In [422]:
G = parse_edges('../DATA/jazz.graph')
n = G.num_vertices
vertices = G.get_vertices()
opt_num = n
opt_cover = vertices
def Branch_and_Bound(cover_set,Gprime,explored,opt_num,opt_cover):
    
    v = choose_vertex(Gprime,explored)
    print('explored',explored)
    if not v:
#         print('no v found')
        return
    v_neighbors = Gprime.get_vertex(v).get_connections()

    Gprime.remove_vertex(v)
    cover_set.add(v)
    
    if not Gprime.get_vertices():
        # solution found
        print('solution found')
        if len(cover_set) < opt_num:
            opt_cover = cover_set
            opt_num = len(cover_set)
            print('optimal:', opt_num)
            
    lower_bound = len(cover_set) + approx2(Gprime) / 2
    if lower_bound < opt_num:
        print('get here')
        Branch_and_Bound(cover_set, Gprime, explored,opt_num,opt_cover)
    
    print('v_neighbors:', v_neighbors)
    for vertex in v_neighbors:
        Gprime.add_edge(v,vertex.id)
        print('graph added vertices')
        
    cover_set.remove(v)
#     explored.add(v)
    lower_bound = len(cover_set) + approx2(Gprime) / 2
    if lower_bound < opt_num:
        Branch_and_Bound(cover_set, Gprime, explored + [v],opt_num,opt_cover)
    
#     return opt_num, opt_cover
    

In [423]:
def choose_vertex(G,explored):
    for vertex in sorted(G.get_vertices(), key = lambda x: -len(G.get_vertex(x).get_connections())):
        if vertex not in explored:
            return vertex

In [683]:
import time
import sys
# from graph import graph # my own code for graph object
import heapq
from collections import deque,defaultdict
import argparse

class vertex:
    def __init__(self,node):
        self.id = node
        self.adjacent = set()

    def __str__(self):
        # for print out result
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor):
    
        self.adjacent.add(neighbor)

    def remove_neighbor(self, neighbor):
        if neighbor in self.adjacent:
            self.adjacent.remove(neighbor)

    def is_connected(self,neighbor):
        return neighbor in self.adjacent

    def get_connections(self):
        return self.adjacent



class graph:
    # unweighted undirected graph
    # can be connected or not
    def __init__(self):
        self.vert_dict = {} # vertex_id (int) : vertex
        self.num_vertices = 0
        self.num_edges = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self,node):
        self.num_vertices += 1
        new_vertex = vertex(node)
        self.vert_dict[node] = new_vertex

    def get_vertex(self,node):
        if node in self.vert_dict:
            return self.vert_dict[node]
        else:
            return None

    def add_edge(self, frm, to):
        # for new vertices
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        if not self.vert_dict[frm].is_connected(self.vert_dict[to]):
            self.num_edges += 1

        self.vert_dict[frm].add_neighbor(self.vert_dict[to])
        self.vert_dict[to].add_neighbor(self.vert_dict[frm])


    def remove_edge(self, frm, to):
        self.vert_dict[frm].remove_neighbor(self.vert_dict[to])
        self.vert_dict[to].remove_neighbor(self.vert_dict[frm])
        self.num_edges -= 1
    
    def remove_vertex(self,node,inplace = False):
        
        G = self.copy() if not inplace else self
        
        for neighbor in list(G.vert_dict[node].get_connections()):
            G.remove_edge(node, neighbor.id)
            if not neighbor.get_connections():
                del G.vert_dict[neighbor.id]
        del G.vert_dict[node]
        
        return G
            

    def get_vertices(self):
        # return a list of ints, the id of vertices
        return list(self.vert_dict.keys())
    
    def get_vertex_degree(self,node):
        return len(self.get_vertex(node).adjacent)
    
    def get_max_degree_vertex(self):
        return max(self.vert_dict, key = lambda x: len(self.get_vertex(x).get_connections()))
    
    def copy(self):
        copy = graph()
        for v in self.vert_dict:
            for u in self.vert_dict[v].get_connections():
                copy.add_edge(v,u.id)
        return copy


        # read data and construct a graph object
def parse_edges(filename):
    # parse edges from graph file to create your graph object
    # filename: string of the filename
    f = open(filename, "r")
    n_vertices, n_edges, _ = f.readline().split(' ')
    n_vertices, n_edges = int(n_vertices), int(n_edges)

    G = graph()  # create a graph

    # add edges to the graph
    for i in range(1, n_vertices + 1):
        neighbors = f.readline().rstrip().split(' ')
        for neighbor in neighbors:
            if neighbor != '':
                if int(neighbor) > i:
                    G.add_edge(i, int(neighbor))
    return G 

def approx2(G):
    # G: a graph object
    # return: the number of vertices of a 2-approximation solution to the min vertex cover problem of G

    S = set()
    for v in G:
        if v in S:
            continue
        for neighbor in v.get_connections():
            
            if neighbor.id < v.id:
                continue
            if v.id not in S and neighbor.id not in S:
                S.add(v.id)
                S.add(neighbor.id)
    return len(S) # dividing the result by 2 gives us a lower bound of the minimum vertex cover of G


def Find_Vprime(G, Cprime):
    'Cprime: a list of numbers in {1, 2, ..., n}'
    'Return: a set of vertices Vprime of the graph Gprime'
    Vprime = set()
#     Cprime = set(Cprime)
#     n = G.num_vertices
    for node in G.get_vertices():
        if node in Cprime:
            continue
            
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id not in Cprime:
                Vprime.add(node)
                break
    return Vprime


def Gprime(G,Vprime):
    'Construct graph Gprime from Vprime'
    G_prime = graph()
    for node in Vprime:
        for neighbor in G.get_vertex(node).adjacent:
            if neighbor.id > node and neighbor.id in Vprime:
                G_prime.add_edge(node,neighbor.id)
    return G_prime

def find_next(V_prime, G_prime, Explored):
    for v in sorted(V_prime, key = lambda x: - G_prime.get_vertex_degree(x)):
        if v not in Explored:
            return v


class node:
    'decision node'
    def __init__(self,id, parent,lb,state):

        self.id = id # the id of the vertex in the original graph G
        self.parent = parent # parent of the current decision node
        self.lb = lb # lower bound of the current decision node 
        self.state = state # state == 1 if we consider including vertex(id) in the vertex cover
    
    def __lt__(self, other): 
        return self.lb > other.lb
         

def Branch_and_Bound(G, start_time, cutoff):
    """
    Find min vertex using a branch and bound algorithm.

    Consider all possible vertex covers and prune nonpromising ones using lower bounds and upper bounds

    Input: a networkx undirected graph
    Returns: the minimum vertex cover
    """
    n = G.num_vertices   
    vertices = G.get_vertices()
    # sort vertices in degree 
    vertices.sort(key = lambda x: -len(G.get_vertex(x).adjacent)) 

    opt_cover = vertices
    opt_num = n

    # initial lowerbound
    LB = approx2(G) / 2   

    # to get the order of vertex using vertex id
    vertices_order = {vertex: i for i, vertex in enumerate(vertices)}

    # number of uncovered edges
    num_uncov_edges0 = G.num_edges # not selecting the vertex[vertices_order[0]]
    num_uncov_edges1 = G.num_edges - G.get_vertex_degree(vertices[0]) # selecting the vertex[vertices_order[0]]

    # initialize priority queue with the first two decision nodes and their corresponding number of uncovered edges
    pqueue = [(num_uncov_edges0, node(vertices[0],None,LB,0)),(num_uncov_edges1,node(vertices[0],None,LB,1))]
    heapq.heapify(pqueue)

    while pqueue and ((time.time() - start_time) < cutoff):

        # get the most promising decision node with the least number of uncovered edges, then the highest lower bound
        num_uncov_edges,Dnode = heapq.heappop(pqueue)

        parent = Dnode.parent
        node_id = Dnode.id   
            
        # initialzie cover set Cprime and explored vertices     
        Cprime = set([Dnode.id]) if Dnode.state else set() # vertices in the conver set
        explored = set() if Dnode.state else set([Dnode.id]) # vertices explored but not in the cover set

        # trace back the decision tree to find Cprime and explored
        while parent:
    
            if parent.state:
                Cprime.add(parent.id)
            else:
                explored.add(parent.id)

            parent = parent.parent
        
        # find V_prime from Cprime
        V_prime = Find_Vprime(G,Cprime)
        # construct G_prime from V_prime
        G_prime = Gprime(G, V_prime)
        
        # find the vertex with the highest degree in G_prime that has not been explored
        new_node_id = find_next(V_prime, G_prime, explored)
        

        if not new_node_id:
            continue # prune it

        
        # Branch 1: add vertex(new_node_id) to the cover set

        cover_size = len(Cprime) + 1  
        new_cover = Cprime.union(set([new_node_id]))
    
        G_prime = G_prime.remove_vertex(new_node_id,inplace = True)
        V_prime = G_prime.get_vertices()
   
        if not V_prime:
            # solution found
            if cover_size < opt_num:
                opt_num = cover_size
                opt_cover = new_cover 
                print('Optimal:', opt_num)
                continue

        
        LowerBound = cover_size + approx2(G_prime)//2

        if LowerBound < opt_num:
            new_node1 = node(new_node_id, Dnode, LowerBound,1)
            heapq.heappush(pqueue, (G_prime.num_edges, new_node1))
    

        # Branch 2: don't add vertex(new_node_id) to the cover set
        if Dnode.lb < opt_num:
            new_node0 = node(new_node_id, Dnode, Dnode.lb,0)
            heapq.heappush(pqueue, (num_uncov_edges, new_node0))

    return opt_num, opt_cover


def main(graph, algo, cutoff, seed):
    G = parse_edges(graph)
    graph_name = graph.split('/')[-1].split('.')[0]
    sol_file = "_".join([graph_name, algo, str(cutoff)]) + '.sol'
    trace_file = "_".join([graph_name, algo, str(cutoff)]) + '.trace'
    output_dir = '../output/'

    start_time = time.time()

    if algo == 'BnB':
        num_vc_nodes, vc = Branch_and_Bound(G, start_time, cutoff)


    total_time = round((time.time() - start_time), 5)
    print('BnB Algo Runtime: ' + str(total_time))

    # Create output directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(os.path.join(output_dir, sol_file), 'w') as f:
        f.write(str(num_vc_nodes) + "\n")
        f.write(','.join([str(n) for n in sorted(vc)]))
    f.close()

    with open(os.path.join(output_dir, trace_file), 'w') as f:
        f.write(' '.join([str(total_time), str(num_vc_nodes)]))
    f.close()

# Run as executable from terminal
# if __name__ == '__main__':
#     #parse arguments in the following format: python code/approx.py -inst DATA/jazz.graph -alg approx -time 600 -seed 30
#     parser = argparse.ArgumentParser(description='Run algorithm with specified parameters')
#     parser.add_argument('-inst', type=str, required=True, help='graph file')
#     parser.add_argument('-alg', type=str, required=True, help='algorithm to use')
#     parser.add_argument('-time', type=float, default=600, required=False, help='runtime cutoff for algorithm')
#     parser.add_argument('-seed', type=int, default=30, required=False, help='random seed for algorithm')
#     args = parser.parse_args()

#     main(args.inst, args.alg, args.time, args.seed)

        

In [684]:
main('../DATA/netscience.graph', 'BnB', 60, 30)

Optimal: 899
BnB Algo Runtime: 60.00075
